In [168]:
import pandas as pd

In [169]:
stop_times = pd.read_csv("/home/invicta117/Downloads/gtfs/stop_times.txt", delimiter=",")
stops = pd.read_csv("/home/invicta117/Downloads/gtfs/stops.txt", delimiter=",")
routes = pd.read_csv("/home/invicta117/Downloads/gtfs/routes.txt", delimiter=",")
trips = pd.read_csv("/home/invicta117/Downloads/gtfs/trips.txt", delimiter=",")

In [170]:
stop_times["arrival_time"] = pd.to_timedelta(stop_times["arrival_time"])

In [167]:
departure = "16:02"
origin = "Clonskeagh Road, stop 878"
destination = "Dawson Street, stop 793"
day = "tuesday"
bus_route = "11"

In [73]:
route = set(trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))]["trip_id"].values)

In [78]:
start = stops[stops["stop_name"] == origin]["stop_id"].values[0]
end = stops[stops["stop_name"] == destination]["stop_id"].values[0]

In [30]:
start

'8250DB000878'

In [159]:
end

'8220DB000793'

In [32]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,5242.y1005.60-1-d12-1.1.O,21:00:00,21:00:00,8240DB000226,1,Sandymount,0,0,0.00
1,5242.y1005.60-1-d12-1.1.O,21:00:38,21:00:38,8220DB000228,2,Sandymount,0,0,267.48
2,5242.y1005.60-1-d12-1.1.O,21:01:11,21:01:11,8240DB000229,3,Sandymount,0,0,483.53
3,5242.y1005.60-1-d12-1.1.O,21:02:02,21:02:02,8240DB000227,4,Sandymount,0,0,834.47
4,5242.y1005.60-1-d12-1.1.O,21:02:35,21:02:35,8240DB000230,5,Sandymount,0,0,1063.49
...,...,...,...,...,...,...,...,...,...
1627380,6667.y1003.60-H9-b12-1.97.I,08:03:12,08:03:12,8220DB000619,16,Abbey Street,0,0,4745.60
1627381,6667.y1003.60-H9-b12-1.97.I,08:03:50,08:03:50,8220DB000675,17,Abbey Street,0,0,4900.43
1627382,6667.y1003.60-H9-b12-1.97.I,08:05:22,08:05:22,8220DB000620,18,Abbey Street,0,0,5277.76
1627383,6667.y1003.60-H9-b12-1.97.I,08:06:35,08:06:35,8220DB007569,19,Abbey Street,0,0,5497.96


In [74]:
start_trips  = set(stop_times[stop_times["stop_id"] == start]["trip_id"].values)
end_trips  = set(stop_times[stop_times["stop_id"] == end]["trip_id"].values)
start_trips = start_trips.intersection(end_trips)
start_trips = start_trips.intersection(route)

In [75]:
start_trips

{'14016.y1004.60-11-b12-1.18.I',
 '14018.y1004.60-11-b12-1.18.I',
 '14020.y1004.60-11-b12-1.18.I',
 '14022.y1004.60-11-b12-1.18.I',
 '14024.y1004.60-11-b12-1.18.I',
 '14026.y1004.60-11-b12-1.18.I',
 '14028.y1004.60-11-b12-1.18.I',
 '14030.y1004.60-11-b12-1.18.I',
 '14032.y1004.60-11-b12-1.18.I',
 '14034.y1004.60-11-b12-1.18.I',
 '14036.y1004.60-11-b12-1.18.I',
 '14038.y1004.60-11-b12-1.18.I',
 '14040.y1004.60-11-b12-1.18.I',
 '14042.y1004.60-11-b12-1.18.I',
 '14044.y1004.60-11-b12-1.18.I',
 '14046.y1004.60-11-b12-1.18.I',
 '14048.y1004.60-11-b12-1.18.I',
 '14050.y1004.60-11-b12-1.18.I',
 '14052.y1004.60-11-b12-1.18.I',
 '14054.y1004.60-11-b12-1.18.I',
 '14056.y1004.60-11-b12-1.18.I',
 '14058.y1004.60-11-b12-1.19.I',
 '14059.y1004.60-11-b12-1.18.I',
 '14061.y1004.60-11-b12-1.18.I',
 '14063.y1004.60-11-b12-1.18.I',
 '14065.y1004.60-11-b12-1.18.I',
 '14066.y1004.60-11-b12-1.18.I',
 '14068.y1004.60-11-b12-1.18.I',
 '14070.y1004.60-11-b12-1.18.I',
 '14072.y1004.60-11-b12-1.19.I',
 '1916.y10

In [109]:
departure

'16:02'

In [151]:
correct_direction = []
for trip in start_trips:
    if (stop_times[stop_times["trip_id"] == trip][stop_times["stop_id"] == start]["stop_sequence"].values[0]) < (stop_times[stop_times["trip_id"] == trip][stop_times["stop_id"] == end]["stop_sequence"].values[0]):
        correct_direction.append(trip)

<ipython-input-151-dfd8c27ba327>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if (stop_times[stop_times["trip_id"] == trip][stop_times["stop_id"] == start]["stop_sequence"].values[0]) < (stop_times[stop_times["trip_id"] == trip][stop_times["stop_id"] == end]["stop_sequence"].values[0]):


In [153]:
len(correct_direction)

186

In [154]:
final = stop_times[stop_times["trip_id"].isin(correct_direction)][stop_times["stop_id"] == start][["trip_id","arrival_time"]]

<ipython-input-154-5864fc062e5d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final = stop_times[stop_times["trip_id"].isin(correct_direction)][stop_times["stop_id"] == start][["trip_id","arrival_time"]]


In [155]:
final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure+':00'))

In [156]:
final = final.sort_values(by="arrival_time")

In [158]:
final

,trip_id,arrival_time
709429,1991.y1003.60-11-b12-1.18.I,0 days 00:00:07
17898,2091.y1005.60-11-d12-1.16.I,0 days 00:00:07
709618,14066.y1004.60-11-b12-1.18.I,0 days 00:02:21
710059,8929.y1005.60-11-b12-1.18.I,0 days 00:14:10
17268,8961.y1006.60-11-d12-1.16.I,0 days 00:14:10
...,...,...
714784,2001.y1003.60-11-b12-1.18.I,0 days 08:59:57
714721,1930.y1003.60-11-b12-1.18.I,0 days 09:19:57
17646,2030.y1005.60-11-d12-1.16.I,0 days 09:19:57
15000,8771.y1006.60-11-d12-1.16.I,0 days 09:21:45


In [160]:
tripid = final.iloc[0]["trip_id"]

In [174]:
seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")

In [175]:
seq

,stop_id,stop_sequence
709408,8250DB000449,1
709409,8250DB000450,2
709410,8250DB003181,3
709411,8250DB000451,4
709412,8250DB000447,5
...,...,...
709466,8220DB000031,62
709467,8220DB000032,63
709468,8220DB000033,64
709469,8220DB007270,65


In [184]:
seq[seq["stop_id"] == start].index[0]

709429

In [185]:
seq[seq["stop_id"] == end].index[0]

709446

In [200]:
test = seq.loc[709429:709446]

In [201]:
test.merge(stops, how='left', on='stop_id')

,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,8250DB000878,22,"Clonskeagh Road, stop 878",53.308153,-6.237091
1,8250DB000879,23,"Annsbrook, stop 879",53.309504,-6.237875
2,8250DB000880,24,"Richview Office Park, stop 880",53.312314,-6.238840
3,8220DB000881,25,"Clonskeagh, stop 881",53.314564,-6.238417
4,8220DB000882,26,"Clonskeagh Hospital, stop 882",53.316705,-6.239155
5,8220DB000883,27,"Eglinton Road, stop 883",53.317685,-6.241036
6,8220DB000884,28,"Norwood Park, stop 884",53.319997,-6.244213
7,8220DB000885,29,"Hollybank Ave Lower, stop 885",53.321259,-6.246293
8,8220DB000886,30,"Merton Drive, stop 886",53.322597,-6.248685
9,8220DB000887,31,"Ranelagh, stop 887",53.324208,-6.252462


,stop_id,stop_name,stop_lat,stop_lon
462,8220DB000786,"Embassy of Malta, stop 786",53.335147,-6.256174
466,8220DB000793,"Dawson Street, stop 793",53.340860,-6.258404
527,8220DB000881,"Clonskeagh, stop 881",53.314564,-6.238417
528,8220DB000882,"Clonskeagh Hospital, stop 882",53.316705,-6.239155
529,8220DB000883,"Eglinton Road, stop 883",53.317685,-6.241036
530,8220DB000884,"Norwood Park, stop 884",53.319997,-6.244213
531,8220DB000885,"Hollybank Ave Lower, stop 885",53.321259,-6.246293
532,8220DB000886,"Merton Drive, stop 886",53.322597,-6.248685
533,8220DB000887,"Ranelagh, stop 887",53.324208,-6.252462
540,8220DB000903,"Chelmsford Road, stop 903",53.325583,-6.252496
